### Loading the model

In [1]:
import pickle
import numpy as np
import pandas as pd
import json
import os

In [3]:
model = pickle.load(open("../model_train/model.pkl","rb"))
model

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10,

Predicting target='success' with two X features: 'energy' and 'danceability'

In [4]:
# Testing known succesful song. It correctly predicts it

print(model.predict([[0.878, 0.619]]))

[1.]


In [5]:
# Testing known unsuccesful song. It correctly predicts it

print(model.predict([[0.583, 0.769]]))

[0.]


Let's use Spotify Web API to get audio features of a new song and predicting its success with the trained model

In [7]:
# Reading Spotify web API credentials from settings.env hidden file

with open('../settings.env') as f:
    env_vars = json.loads(f.read())

In [8]:
# Set environment variables
os.environ['SPOTIPY_CLIENT_ID'] = env_vars['SPOTIPY_CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET'] = env_vars['SPOTIPY_CLIENT_SECRET']

In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

query = 'you oughta know'

search = sp.search(q = query, type='track', market='US')

#print('artist: ',search['tracks']['items'][0]['artists'][0]['name'], 
#      '\nsong: ',search['tracks']['items'][0]['name'],
#      '\nalbum: ',search['tracks']['items'][0]['album']['name'],
#      '\ntrack_id: ',search['tracks']['items'][0]['id'])

In [16]:
print('artist: ',search['tracks']['items'][0]['artists'][0]['name'], 
      '\nsong: ',search['tracks']['items'][0]['name'],
      '\nalbum: ',search['tracks']['items'][0]['album']['name'],
      '\ntrack_id: ',search['tracks']['items'][0]['id'])

track_id = search['tracks']['items'][0]['id']

artist:  Alanis Morissette 
song:  You Oughta Know - 2015 Remaster 
album:  Jagged Little Pill (2015 Remaster) 
track_id:  3jS7bB0oXVOwGFZn3aE5NV


Now let's find the values of the first two audio_features (danceability and energy) for that song

In [25]:
audio_features = sp.audio_features(track_id)
audio_features

[{'danceability': 0.665,
  'energy': 0.834,
  'key': 4,
  'loudness': -7.737,
  'mode': 1,
  'speechiness': 0.0576,
  'acousticness': 0.21,
  'instrumentalness': 0,
  'liveness': 0.452,
  'valence': 0.411,
  'tempo': 105.292,
  'type': 'audio_features',
  'id': '3jS7bB0oXVOwGFZn3aE5NV',
  'uri': 'spotify:track:3jS7bB0oXVOwGFZn3aE5NV',
  'track_href': 'https://api.spotify.com/v1/tracks/3jS7bB0oXVOwGFZn3aE5NV',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3jS7bB0oXVOwGFZn3aE5NV',
  'duration_ms': 249493,
  'time_signature': 4}]

In [41]:
print(audio_features[0]['danceability'])
print(audio_features[0]['energy'])

0.719
0.704


Now let's find the prediction for the track with those features

In [27]:
print(model.predict([[audio_features[0]['danceability'], audio_features[0]['energy']]]))

[0.]


The result for 'You Oughta Know', a hit track from 1996, is 0, unsuccessful, but we know we have to improve the model prototype as it only evaluates 100 succesful songs.

Let's predict the success of a hit song that is already in the model

In [38]:
# Trying to condense the query in one search line

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

query = 'sweet but psycho'

audio_features = sp.audio_features((sp.search(q = query, type='track', market='US'))['tracks']['items'][0]['id'])

In [40]:
print(model.predict([[audio_features[0]['danceability'], audio_features[0]['energy']]]))

[1.]
